In [1]:
import pandas as pd

#First, we read the data

data_rates=pd.read_csv("/Users/jsjaramillod/Desktop/IMA_bootcamp/Project/MacroData/yield-curve-rates-2004-2024.csv")

#copy
rates_df=data_rates.copy()

In [2]:
#Second we modify the format, this is the solution that I found for the two digit year
def chain_date(date_str):
    for fmt in ('%m/%d/%Y', '%m/%d/%y'):
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            pass
    return pd.NaT

# We apply the chain_date function
rates_df['Date'] = rates_df['Date'].apply(chain_date)


copy_df=rates_df.copy()

In [3]:
# Assign quarter
def assign_quarter(date):
    month = date.month
    if month in [1, 2, 3]:
        return 'Q1'
    elif month in [4, 5, 6]:
        return 'Q2'
    elif month in [7, 8, 9]:
        return 'Q3'
    else:
        return 'Q4'

# Asisign the quarters in a new column
copy_df['Quarter'] = copy_df['Date'].apply(assign_quarter)
copy_df['Year'] = copy_df['Date'].dt.year


# Agrupar por cuatrimestre y calcular el promedio de todas las columnas excepto 'Date'
quarterly_averages = copy_df.groupby(['Year', 'Quarter']).mean(numeric_only=True).reset_index()



In [4]:
import itertools

#calculate the spreads for all the possible spreads (around 78 possible)
columns_to_consider = quarterly_averages.columns[2:]


column_pairs = list(itertools.combinations(columns_to_consider, 2))


#new data frame to store the spreads
spreads_df = pd.DataFrame()

for (col1, col2) in column_pairs:
    spread_col_name = f"{col2}-{col1}_spread"
    spreads_df[spread_col_name] = quarterly_averages[col2] - quarterly_averages[col1]
    
    
#Add the year-Quarter columns must have 80 columns in total
result_df = pd.concat([quarterly_averages.iloc[:, :2], spreads_df], axis=1)

result_df.head()

,Year,Quarter,2 Mo-1 Mo_spread,3 Mo-1 Mo_spread,4 Mo-1 Mo_spread,6 Mo-1 Mo_spread,1 Yr-1 Mo_spread,2 Yr-1 Mo_spread,3 Yr-1 Mo_spread,5 Yr-1 Mo_spread,...,7 Yr-5 Yr_spread,10 Yr-5 Yr_spread,20 Yr-5 Yr_spread,30 Yr-5 Yr_spread,10 Yr-7 Yr_spread,20 Yr-7 Yr_spread,30 Yr-7 Yr_spread,20 Yr-10 Yr_spread,30 Yr-10 Yr_spread,30 Yr-20 Yr_spread
0,2004,Q1,NaN,0.024032,NaN,0.093871,0.311774,0.776935,1.253387,2.071290,...,0.520484,1.029032,1.896935,NaN,0.508548,1.376452,NaN,0.867903,NaN,NaN
1,2004,Q2,NaN,0.141613,NaN,0.405161,0.820968,1.500161,2.017419,2.766452,...,0.462903,0.876129,1.634677,NaN,0.413226,1.171774,NaN,0.758548,NaN,NaN
2,2004,Q3,NaN,0.143906,NaN,0.418437,0.705000,1.186562,1.549531,2.135625,...,0.416562,0.795781,1.562656,NaN,0.379219,1.146094,NaN,0.766875,NaN,NaN
3,2004,Q4,NaN,0.205645,NaN,0.455806,0.630968,0.980645,1.217097,1.651935,...,0.361290,0.682097,1.381129,NaN,0.320806,1.019839,NaN,0.699032,NaN,NaN
4,2005,Q1,NaN,0.228197,NaN,0.515902,0.711475,1.093934,1.262951,1.529836,...,0.208033,0.412787,0.873279,NaN,0.204754,0.665246,NaN,0.460492,NaN,NaN
